# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
#!jupyter nbextension enable --py --sys-prefix gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
citi_p1=pd.read_csv("../WeatherPy/WeatherPy_Database.csv", encoding="utf-8")
citi_p1


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,70.81,56,54,2.57,broken clouds
1,1,Loa Janan,ID,-0.5830,117.0950,75.70,96,97,0.16,overcast clouds
2,2,Yellowknife,CA,62.4560,-114.3525,36.12,51,20,10.36,few clouds
3,3,Gilgit,PK,35.9221,74.3087,75.70,41,96,5.21,overcast clouds
4,4,Esperance,AU,-33.8667,121.9000,63.10,64,100,14.38,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
576,576,Tingo Maria,PE,-9.2895,-76.0088,80.91,74,40,4.61,scattered clouds
577,577,Nemuro,JP,43.3236,145.5750,38.64,94,61,6.26,broken clouds
578,578,Axim,GH,4.8699,-2.2405,84.29,72,100,7.78,overcast clouds
579,579,Jedrzejow,PL,50.6394,20.3045,68.14,28,42,1.99,scattered clouds


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
#gmaps.configure()

In [13]:
frizzy=citi_p1["Humidity"].astype(float)
maxfrizz=frizzy.max()
places=citi_p1[["Lat","Lng"]]

fig=gmaps.figure()
friz_hm=gmaps.heatmap_layer(places, weights=frizzy, dissipating=False, max_intensity=maxfrizz,point_radius=3)
fig.add_layer(friz_hm)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
vaca_citi_df=citi_p1.loc[(citi_p1["Max Temp"] > 70) & (citi_p1["Max Temp"] < 80) & (citi_p1["Cloudiness"] == 0), :]
vaca_citi_df=vaca_citi_df.dropna(how='any')
vaca_citi_df.reset_index(inplace=True)
del vaca_citi_df['index']
vaca_citi_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,33,Carnarvon,AU,-24.8667,113.6333,73.40,67,0,20.80,clear sky
1,34,Jalu,LY,29.0331,21.5482,75.27,40,0,27.34,clear sky
2,105,Mayo,US,38.8876,-76.5119,75.13,72,0,4.61,haze
3,133,Saint-Pierre,RE,-21.3393,55.4781,74.88,82,0,4.61,clear sky
4,142,Beckley,US,37.7782,-81.1882,72.09,44,0,11.50,clear sky


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
gmaps.configure(api_key=g_key)
hotels = []

for city in range(len(vaca_citi_df["City"])):
    lat = vaca_citi_df.loc[city]["Lat"]
    lng = vaca_citi_df.loc[city]["Lng"]
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "lodging",
        "key" : g_key
    }
    
    b_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    req = requests.get(b_url, params=params)
    jsn = req.json()
    
    try: 
        hotels.append(jsn["results"][0]["name"])
    except:
        hotels.append("Hotel not found")
        
vaca_citi_df["Hotel Name"] = hotels
vaca_citi_df = vaca_citi_df.dropna(how='any')
vaca_citi_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Hotel Name
0,33,Carnarvon,AU,-24.8667,113.6333,73.40,67,0,20.80,clear sky,Hospitality Carnarvon
1,34,Jalu,LY,29.0331,21.5482,75.27,40,0,27.34,clear sky,صاحبة الفخامة
2,105,Mayo,US,38.8876,-76.5119,75.13,72,0,4.61,haze,Resorts Resource Group International
3,133,Saint-Pierre,RE,-21.3393,55.4781,74.88,82,0,4.61,clear sky,Alize Plage
4,142,Beckley,US,37.7782,-81.1882,72.09,44,0,11.50,clear sky,"Country Inn & Suites by Radisson, Beckley, WV"
5,303,Buin,CL,-33.7333,-70.7500,71.87,38,0,1.05,clear sky,Paihuen
6,339,Las Cruces,US,32.3123,-106.7783,76.05,18,0,20.71,clear sky,SpringHill Suites by Marriott Las Cruces
7,388,Mopipi,BW,-21.1833,24.8833,72.99,23,0,7.38,clear sky,Hotel not found
8,435,Makakilo City,US,21.3469,-158.0858,72.19,79,0,9.22,clear sky,Marriott's Ko Olina Beach Club
9,455,Urumqi,CN,43.8010,87.6005,76.15,13,0,20.13,clear sky,Hua Ling Grand Hotel Urumqi


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vaca_citi_df.iterrows()]
locations = vaca_citi_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
mrkrs= gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
fig.add_layer(mrkrs)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))